In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import sys, os
sys.path.append("../scripts")

import logging
logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel('INFO')

In [2]:
# load the data
genotypes = pd.read_csv("X.csv", index_col=0)
phenotypes = pd.read_csv("phenotypes.csv", index_col=0)
snp_locations = pd.read_csv("mice_map2.csv")
marker_locations = pd.read_csv("MGI_MRK_Coord.rpt.csv")
marker_locations.Chromosome = marker_locations.Chromosome.astype(str)

logger.info("genotypes shape is {}, phenotypes shape is {}".format(genotypes.shape, phenotypes.shape))

/home/jsi17/anaconda3/envs/tf2-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
INFO:__main__:genotypes shape is (1814, 10346), phenotypes shape is (1814, 6)


In [3]:
marker_locations

,MGI Marker Accession ID,Marker Type,Feature Type,Marker Symbol,Marker Name,Chromosome,Start Coordinate,End Coordinate,Strand,Genome Build,Provider Collection,Provider Display
0,MGI:87860,Gene,protein coding gene,Abl2,v-abl Abelson murine leukemia viral oncogene 2...,1,156558786,156649568,+,GRCm38,Ensembl Gene Model,Ensembl
1,MGI:87866,Gene,protein coding gene,Acadl,"acyl-Coenzyme A dehydrogenase, long-chain",1,66830839,66863277,-,GRCm38,Ensembl Gene Model,Ensembl
2,MGI:87893,Gene,protein coding gene,Chrnd,"cholinergic receptor, nicotinic, delta polypep...",1,87190607,87200070,+,GRCm38,Ensembl Gene Model,Ensembl
3,MGI:87895,Gene,protein coding gene,Chrng,"cholinergic receptor, nicotinic, gamma polypep...",1,87205811,87211835,+,GRCm38,NCBI Gene Model,NCBI
4,MGI:87948,Gene,protein coding gene,Adss,"adenylosuccinate synthetase, non muscle",1,177763176,177796695,-,GRCm38,NCBI Gene Model,NCBI
...,...,...,...,...,...,...,...,...,...,...,...,...
265365,MGI:6092513,Other Genome Feature,TSS region,Tssr164746,transcription start site region 164746,Y,4225995,4226013,-,GRCm38,RIKEN,RIKEN
265366,MGI:6092514,Other Genome Feature,TSS region,Tssr164747,transcription start site region 164747,Y,4226100,4226106,-,GRCm38,RIKEN,RIKEN
265367,MGI:6092515,Other Genome Feature,TSS region,Tssr164748,transcription start site region 164748,Y,4227178,4227185,-,GRCm38,RIKEN,RIKEN
265368,MGI:6096127,Gene,miRNA gene,Gm47281,"predicted gene, 47281",Y,991657,991716,-,GRCm38,Ensembl Gene Model,Ensembl


In [4]:
# some formatting of the genotypes
new_snp_names = [x[0] for x in genotypes.columns.str.split("_").str[:1]]
genotypes.columns = new_snp_names

snps_with_locs = np.isin(genotypes.columns, snp_locations.marker)
logger.info(f"{snps_with_locs.sum()} of {genotypes.shape[1]} genotypes have locations")

genotypes = genotypes.loc[:,snps_with_locs]
snp_locations = snp_locations.loc[ snp_locations.marker.isin(genotypes.columns) ].reset_index(drop=True)

INFO:__main__:8854 of 10346 genotypes have locations


In [ ]:
# map the SNPs to the markers (Genes) by iterating over the SNPs
# and seeing if they fall inside the marker boundaries
# this is very slow, takes around 10 minutes on my machine
# SNPs can be on multiple markers as they overlap
snp2marker = {}

for _, snp_row in snp_locations.iterrows():
    markers_on_chromosome = marker_locations.loc[ marker_locations.Chromosome==snp_row.chromosome ]
    row_indexer = np.logical_and(
        markers_on_chromosome["Start Coordinate"] < snp_row.bp,
        markers_on_chromosome["End Coordinate"] > snp_row.bp
    )
    markers_matching_snp = markers_on_chromosome.loc[row_indexer,:]
    if markers_matching_snp.shape[0] > 1:
        snp2marker[snp_row.marker] = markers_matching_snp["Marker Symbol"].to_numpy().tolist()

In [8]:
snp_dict_map = pd.concat(
    {k: pd.Series(v) for k, v in snp2marker.items()}
).reset_index().rename(
    columns={'level_0' : 'snp', 0 : 'Marker'}
).drop(
    columns="level_1"
)
snp_dict_map

,snp,Marker
0,rs3707673,Xkr4
1,rs3707673,Gm37180
2,rs3707673,Gm39585
3,rs3707673,Ebas2
4,rs3707673,Twq5
...,...,...
121993,gnfX.148.995,Obsty5
121994,rs13484113,Li
121995,rs13484113,Bw3
121996,rs13484113,Spha3


In [10]:
# some post processing - we are only interested in SNPs on a single
# gene
snps_per_marker = snp_dict_map.groupby("Marker", as_index=False).size()
snps_per_marker.sort_values("size", ascending=False)

# only keep SNPs mapped to genes
genes_only_map = snp_dict_map.loc[ snp_dict_map.Marker.isin(
    marker_locations["Marker Symbol"][ marker_locations["Marker Type"]=="Gene" ]
)].reset_index(drop=True)

# only keep genes with more than one SNP
genes_only_map = genes_only_map.loc[ genes_only_map.Marker.isin(
    snps_per_marker["Marker"][ snps_per_marker["size"]>1 ]
)]

included_snps = genes_only_map.snp.unique()
included_genes = genes_only_map.Marker.unique()

logger.info(f"Keeping {included_snps.shape[0]:,} SNPs and {included_genes.shape[0]} genes")

2022-10-11 15:29:49 INFO     Keeping 5,503 SNPs and 764 genes
